In [44]:
import pandas as pd 
import pickle
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [45]:
data_raw = pd.read_csv("BTC_Tweets_Updated.csv")
data_raw.head(2)

,id,Date,Tweet,Screen_name,Source,Link,Sentiment,sent_score,New_Sentiment_Score,New_Sentiment_State,BERT Labels
0,0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['neutral'],0.0,0.0,0.0,1
1,1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",['neutral'],0.0,0.0,0.0,0


In [46]:
df = data_raw[['Tweet','Sentiment']]
df['Sentiment'] = df['Sentiment'].str.replace(r'\W+', '', regex=True)
df

C:\Users\untu0\AppData\Local\Temp\ipykernel_45892\2011413221.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentiment'] = df['Sentiment'].str.replace(r'\W+', '', regex=True)


,Tweet,Sentiment
0,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",neutral
1,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,neutral
2,RT @tippereconomy: Another use case for #block...,positive
3,free coins https://t.co/DiuoePJdap,positive
4,RT @payvxofficial: WE are happy to announce th...,positive
...,...,...
50847,RT @fixy_app: Fixy Network brings popular cryp...,positive
50848,RT @bethereumteam: After a successful launch o...,positive
50849,"RT @GymRewards: Buy #GYMRewards Tokens, Bonus ...",neutral
50850,I added a video to a @YouTube playlist https:/...,positive


In [47]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text):
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Lowercasing
    tokens = [token.lower() for token in tokens]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join tokens back into text
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\untu0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\untu0\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\untu0\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [48]:
df['Tweet'] = df['Tweet'].apply(clean_text)

C:\Users\untu0\AppData\Local\Temp\ipykernel_45892\3768818029.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweet'] = df['Tweet'].apply(clean_text)


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(df['Tweet'])

In [50]:
y=df['Sentiment']

In [51]:
split80 = int(len(df)*0.8)
x_train,x_test = x[:split80], x[split80:]
y_train,y_test = y[:split80], y[split80:]

In [52]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()
model = NB.fit(x_train,y_train)
y_pred1 = model.predict(x_test)


In [53]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

    negative       0.96      0.60      0.74      1145
     neutral       0.88      0.87      0.88      4434
    positive       0.84      0.93      0.89      4592

    accuracy                           0.87     10171
   macro avg       0.90      0.80      0.83     10171
weighted avg       0.87      0.87      0.87     10171



In [54]:
file = open('NB.pickle', 'wb')
pickle.dump(model, file)
file.close()

In [55]:
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)